In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# safest renderer
pio.renderers.default = "iframe"
import numpy as np

### Model acceleration ###
Here we propose a plausible model for the acceleration function and compute its antiderivatives to determine velocity and position.

In [5]:
# acceleration function
def a_vec(t_array):
    t_array = np.asarray(t_array)

    # Define the conditions
    conds = [
        (0 <= t_array) & (t_array < 1),
        (1 <= t_array) & (t_array < 2),
        (2 <= t_array) & (t_array < 3),
        (3 <= t_array) & (t_array < 7),
        (7 <= t_array) & (t_array < 8),
        (8 <= t_array) & (t_array < 9),
        (9 <= t_array) & (t_array <= 10),
    ]

    # Define the corresponding expressions
    funcs = [
        0,
        t_array - 1,
        -t_array + 3,
        0,
        -t_array + 7,
        t_array - 9,
        0
    ]

    # Compute piecewise values
    result = np.select(conds, funcs, default=0)

    return result


In [7]:
# velocity function
def v_vec(t_array):
    t_array = np.asarray(t_array)

    # Define conditions
    conds = [
        (0 <= t_array) & (t_array < 1),
        (1 <= t_array) & (t_array < 2),
        (2 <= t_array) & (t_array < 3),
        (3 <= t_array) & (t_array < 7),
        (7 <= t_array) & (t_array < 8),
        (8 <= t_array) & (t_array < 9),
        (9 <= t_array) & (t_array <= 10),
    ]

    # Define corresponding functions or constants
    funcs = [
        0,
        0.5 * t_array**2 - t_array + 0.5,
        -0.5 * t_array**2 + 3 * t_array - 3.5,
        1,
        -0.5 * t_array**2 + 7 * t_array - 23.5,
        0.5 * t_array**2 - 9 * t_array + 40.5,
        0
    ]

    # Compute the piecewise result
    result = np.select(conds, funcs, default=0)

    return result

In [9]:
# position function
def s_vec(t_array):
    t_array = np.asarray(t_array)

    # Define conditions
    conds = [
        (0 <= t_array) & (t_array < 1),
        (1 <= t_array) & (t_array < 2),
        (2 <= t_array) & (t_array < 3),
        (3 <= t_array) & (t_array < 7),
        (7 <= t_array) & (t_array < 8),
        (8 <= t_array) & (t_array < 9),
        (9 <= t_array) & (t_array <= 10),
    ]

    # Define expressions for each interval
    funcs = [
        0,
        (1/6)*t_array**3 - 0.5*t_array**2 + 0.5*t_array - (1/6),
        -(1/6)*t_array**3 + 1.5*t_array**2 - 3.5*t_array + 2.5,
        t_array - 2,
        -(1/6)*t_array**3 + 3.5*t_array**2 - 23.5*t_array + (331/6),
        (1/6)*t_array**3 - 4.5*t_array**2 + 40.5*t_array - 115.5,
        6
    ]

    result = np.select(conds, funcs, default=0)

    return result

In [11]:
X=np.linspace(0,10,1000)
df = pd.DataFrame(dict(x=X))
df.head()

,x
0,0.00000
1,0.01001
2,0.02002
3,0.03003
4,0.04004


In [13]:
df['a'] = a_vec(df['x'])
df['v'] = v_vec(df['x'])
df['s'] = s_vec(df['x'])

In [15]:
df.head()

,x,a,v,s
0,0.00000,0.0,0.0,0.0
1,0.01001,0.0,0.0,0.0
2,0.02002,0.0,0.0,0.0
3,0.03003,0.0,0.0,0.0
4,0.04004,0.0,0.0,0.0


In [17]:
fig = go.Figure()
# Update layout if desired
fig.update_layout(
    title="Acceleration, Velocity, Position",
    xaxis_title="Time(s)",
    yaxis_title=""
)
# Add the acceleration trace
fig.add_trace(go.Scatter(x=df['x'], y=df['a'], mode='lines', name='Acceleration', line=dict(color='blue')))
# Add the velocity trace
fig.add_trace(go.Scatter(x=df['x'], y=df['v'], mode='lines', name='Velocity', line=dict(color='red')))
# Add the position trace
fig.add_trace(go.Scatter(x=df['x'], y=df['s'], mode='lines', name='Position', line=dict(color='green')))

fig.show()


### Acceleration as data ###
Treat the acceleration column as measured (very clean) data. Use the `.cumsum()` function to compute the velocity and position columns.

In [21]:
X=np.linspace(0,10,1000)
df_data = pd.DataFrame(dict(x=X))
df_data.head()

,x
0,0.00000
1,0.01001
2,0.02002
3,0.03003
4,0.04004


In [23]:
df_data['a'] = a_vec(df_data['x'])
dt = X[1]-X[0]
# compute velocity and position using numerical method
df_data['v'] = df_data['a'].cumsum()*dt
df_data['s'] = df_data['v'].cumsum()*dt

In [25]:
fig_data = go.Figure()
# Update layout if desired
fig_data.update_layout(
    title="Acceleration, Velocity, Position",
    xaxis_title="Time(s)",
    yaxis_title=""
)
# Add the acceleration trace
fig_data.add_trace(go.Scatter(x=df_data['x'], y=df_data['a'], mode='lines', name='Acceleration', line=dict(color='blue')))
# Add the velocity trace
fig_data.add_trace(go.Scatter(x=df_data['x'], y=df_data['v'], mode='lines', name='Velocity', line=dict(color='red')))
# Add the position trace
fig_data.add_trace(go.Scatter(x=df_data['x'], y=df_data['s'], mode='lines', name='Position', line=dict(color='green')))

fig_data.show()

### Acceleration as noisy data ###
Add some normally distributed, zero-mean noise with 0.5 standard deviation to the acceleration data.

In [62]:
X=np.linspace(0,10,1000)
df_data_noise = pd.DataFrame(dict(x=X))
df_data_noise.head()

,x
0,0.00000
1,0.01001
2,0.02002
3,0.03003
4,0.04004


In [82]:
noise = np.random.normal(loc=0.0, scale=0.5, size=len(X))
df_data_noise['a'] = a_vec(df_data_noise['x']) + noise
# compute velocity and position using numerical method
df_data_noise['v'] = df_data_noise['a'].cumsum()*(10.0/999.0)
df_data_noise['s'] = df_data_noise['v'].cumsum()*(10.0/999.0)

In [84]:
fig_data_noise = go.Figure()
# Update layout if desired
fig_data_noise.update_layout(
    title="Acceleration, Velocity, Position + Noise",
    xaxis_title="Time(s)",
    yaxis_title=""
)
# Add the acceleration trace
fig_data_noise.add_trace(go.Scatter(x=df_data_noise['x'], y=df_data_noise['a'], mode='markers', name='Acceleration', line=dict(color='blue')))
# Add the velocity trace
fig_data_noise.add_trace(go.Scatter(x=df_data_noise['x'], y=df_data_noise['v'], mode='lines', name='Velocity', line=dict(color='red')))
# Add the position trace
fig_data_noise.add_trace(go.Scatter(x=df_data_noise['x'], y=df_data_noise['s'], mode='lines', name='Position', line=dict(color='green')))

fig_data_noise.show()